In [1]:
import pandas as pd
import os
import datetime

cpu_undersized = pd.read_csv('data/final_data/cpu_undersized.csv')
cpu_oversized = pd.read_csv('data/final_data/cpu_oversized.csv')
ram_undersized = pd.read_csv('data/final_data/ram_undersized.csv')
ram_oversized = pd.read_csv('data/final_data/ram_oversized.csv')

data_mycloud = pd.read_csv('data/raw_data/mycloud_20221221.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'data/final_data/cpu_undersized.csv'

In [2]:
cpu_undersized = cpu_undersized[['name_server', 'key', 'Price', 'CPU', 'RAM', 'name_environment']]
ram_undersized = ram_undersized[['name_server', 'key', 'Price', 'CPU', 'RAM', 'name_environment']]
cpu_oversized = cpu_oversized[['name_server', 'key', 'Price', 'CPU', 'RAM', 'name_environment']]
ram_oversized = ram_oversized[['name_server', 'key', 'Price', 'CPU', 'RAM', 'name_environment']]

In [3]:
def add_column_CPU_RAM(data_cpu: pd.DataFrame, data_ram: pd.DataFrame) -> pd.DataFrame:
    """
    Indicates if both CPU and RAM are undersized/oversized or not.
    """
    cpu_set = set(data_cpu['name_server'])
    ram_set = set(data_ram['name_server'])
    both_set = list(ram_set.intersection(cpu_set))

    data_cpu['both'] = data_cpu.name_server.apply(lambda name: True if name in both_set else False)
    data_ram['both'] = data_ram.name_server.apply(lambda name: True if name in both_set else False)

    return data_cpu, data_ram


In [4]:
mycloud_configs_cpu_to_ram = {
    1: [2, 4, 6, 8, 12, 16, 24, 32],
    2: [4, 8, 16, 24, 32, 64, 128],
    4: [8, 16, 24, 32, 64, 128],
    8: [16, 24, 32, 64, 128],
    12: [24, 32, 48, 64, 128],
    16: [32, 48, 64, 128]
}

mycloud_configs_ram_to_cpu = {
    2: [1],
    4: [1, 2],
    6: [1],
    8: [1, 2, 4],
    12: [1],
    16: [1, 2, 4, 8],
    24: [1, 2, 4, 8, 12],
    32: [1, 2, 4, 8, 12, 16],
    48: [12, 16],
    64: [2, 4, 8, 12, 16],
    128: [2, 4, 8, 12, 16]
}


def upper_value(num: int, list_values: list, check_same: bool=False) -> int:
    """
    Find upper bound of an int in a list
    """
    if check_same:
        listing = [item for item in list_values if item >= num]
    else:
        listing = [item for item in list_values if item > num]
    listing.sort()
    return listing[0]

def lower_value(num: int, list_values: list, check_same: bool=False) -> int:
    """
    Find upper bound of an int in a list
    """
    if check_same:
        listing = [item for item in list_values if item <= num]
    else:
        listing = [item for item in list_values if item < num]
    listing.sort(reverse=True)
    return listing[0]

def closest(lst, K) -> int:
    """
    Finds closest number in a given list to K
    """
    return lst[min(range(len(lst)), key = lambda i: abs(lst[i]-K))]

In [5]:
def config_change(row, cpu: bool, undersized: bool):
    """
    Change config
    TO DO : à méditer si on doit toujours décroitre simple ou parfois double ET 
    si on doit toujours faire le moove même si ça nous fait par exemple modifier la ram sans nécessité 
    car la config du nv nb de cpu l'oblige (peut même être dangereux sur une baisse de RAM)
    """
    if cpu:
        if undersized:
            if row.both:
                cpu = upper_value(row.CPU, list(mycloud_configs_cpu_to_ram.keys()))
                ram = upper_value(row.RAM, mycloud_configs_cpu_to_ram[cpu])
                return f"{cpu}#{ram}", cpu, ram
            else:
                cpu = upper_value(row.CPU, list(mycloud_configs_cpu_to_ram.keys()))
                ram = upper_value(row.RAM, mycloud_configs_cpu_to_ram[cpu], check_same=True)
                return f"{cpu}#{ram}", cpu, ram
        else:
            if row.both:
                cpu = lower_value(row.CPU, list(mycloud_configs_cpu_to_ram.keys()))
                ram = lower_value(row.RAM, mycloud_configs_cpu_to_ram[cpu])
                return f"{cpu}#{ram}", cpu, ram
            else:
                cpu = lower_value(row.CPU, list(mycloud_configs_cpu_to_ram.keys()))
                ram = lower_value(row.RAM, mycloud_configs_cpu_to_ram[cpu], check_same=True)
                return f"{cpu}#{ram}", cpu, ram
    else:
        if undersized:
            if row.both:
                cpu = upper_value(row.CPU, list(mycloud_configs_cpu_to_ram.keys()))
                ram = upper_value(row.RAM, mycloud_configs_cpu_to_ram[cpu])
                return f"{cpu}#{ram}", cpu, ram
            else:
                ram = upper_value(row.RAM, list(mycloud_configs_ram_to_cpu.keys()))
                try:
                    cpu = upper_value(row.CPU, mycloud_configs_ram_to_cpu[ram], check_same=True)
                except:
                    cpu = closest(mycloud_configs_ram_to_cpu[ram], row.CPU)
                return f"{cpu}#{ram}", cpu, ram
        else:
            if row.both:
                cpu = lower_value(row.CPU, list(mycloud_configs_cpu_to_ram.keys()))
                ram = lower_value(row.RAM, mycloud_configs_cpu_to_ram[cpu])
                return f"{cpu}#{ram}", cpu, ram
            else:
                ram = lower_value(row.RAM, list(mycloud_configs_ram_to_cpu.keys()))
                try:
                    cpu = lower_value(row.CPU, mycloud_configs_ram_to_cpu[ram], check_same=True)
                except:
                    cpu = closest(mycloud_configs_ram_to_cpu[ram], row.CPU)
                return f"{cpu}#{ram}", cpu, ram
    



In [6]:
def add_columns_config_changes(dataset, undersized, cpu) -> pd.DataFrame:
    """
    Add columns change in config
    """
    dataset['new_config'] = dataset.apply(lambda row: config_change(row, cpu=cpu, undersized=undersized)[0], axis=1)
    dataset['new_CPU'] = dataset.apply(lambda row: config_change(row, cpu=cpu, undersized=undersized)[1], axis=1)
    dataset['new_RAM'] = dataset.apply(lambda row: config_change(row, cpu=cpu, undersized=undersized)[2], axis=1)
    if cpu:
        dataset['unnecessary_change_in_ram'] = dataset.apply(lambda row: row.new_RAM - row.RAM,axis=1)
    else:
        dataset['unnecessary_change_in_cpu'] = dataset.apply(lambda row: row.new_CPU - row.CPU,axis=1)

    return dataset

In [7]:
cpu_undersized, ram_undersized = add_column_CPU_RAM(cpu_undersized, ram_undersized)
cpu_undersized = add_columns_config_changes(cpu_undersized, undersized=True, cpu=True)
ram_undersized = add_columns_config_changes(ram_undersized, undersized=True, cpu=False)
cpu_oversized, ram_oversized = add_column_CPU_RAM(cpu_oversized, ram_oversized)
cpu_oversized = add_columns_config_changes(cpu_oversized, undersized=False, cpu=True)
ram_oversized = add_columns_config_changes(ram_oversized, undersized=False, cpu=False)

In [8]:
cpu_u_for_join = cpu_undersized[['name_server', 'key', 'Price', 'new_config']]
cpu_o_for_join = cpu_oversized[['name_server', 'key', 'Price', 'new_config']]
ram_u_for_join = ram_undersized[['name_server', 'key', 'Price', 'new_config']]
ram_o_for_join = ram_oversized[['name_server', 'key', 'Price', 'new_config']]

all_changes = pd.concat([cpu_u_for_join, cpu_o_for_join, ram_u_for_join, ram_o_for_join], axis=0).drop_duplicates(subset='name_server')
all_changes = all_changes.merge(data_mycloud[['key', 'Price']], left_on='new_config', right_on='key', how='left', suffixes=['_old', '_new'])
all_changes['price_delta'] =  all_changes['Price_new'] - all_changes['Price_old']

In [9]:
# Creating the Excel
writer = pd.ExcelWriter("data/excels/change_config.xlsx", engine='xlsxwriter')

cpu_undersized.to_excel(writer, sheet_name='CPU undersized', startrow=1, header=False)
cpu_oversized.to_excel(writer, sheet_name='CPU oversized', startrow=1, header=False)
ram_undersized.to_excel(writer, sheet_name='RAM undersized', startrow=1, header=False)
ram_oversized.to_excel(writer, sheet_name='RAM oversized', startrow=1, header=False)
all_changes.to_excel(writer, sheet_name='Expenses variation', startrow=1, header=False)

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet_CPU_u = writer.sheets['CPU undersized']
worksheet_CPU_o = writer.sheets['CPU oversized']
worksheet_RAM_u = writer.sheets['RAM undersized']
worksheet_RAM_o = writer.sheets['RAM oversized']
worksheet_pricing = writer.sheets['Expenses variation']

# Add a header format.
header_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'top',
    'fg_color': '#D7E4BC',
    'border': 1})

# Write the column headers with the defined format.
for col_num, value in enumerate(cpu_undersized.columns.values):
    worksheet_CPU_u.write(0, col_num + 1, value, header_format)
for col_num, value in enumerate(cpu_oversized.columns.values):
    worksheet_CPU_o.write(0, col_num + 1, value, header_format)
for col_num, value in enumerate(ram_undersized.columns.values):
    worksheet_RAM_u.write(0, col_num + 1, value, header_format)
for col_num, value in enumerate(ram_oversized.columns.values):
    worksheet_RAM_o.write(0, col_num + 1, value, header_format)
for col_num, value in enumerate(all_changes.columns.values):
    worksheet_pricing.write(0, col_num + 1, value, header_format)

# Close the Pandas Excel writer and output the Excel file.
writer.close()